In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data=pd.read_csv('/kaggle/input/drug-classification/drug200.csv')
data

In [ ]:
data.describe()

In [ ]:
pd.crosstab(data['Cholesterol'],data['BP'])

In [ ]:
pd.crosstab(data['Cholesterol'],data['Drug'])

In [ ]:
pd.crosstab(data['Sex'],data['Drug'])

In [ ]:
#check null values in each column
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
#check null values in each columnmissing_data.head()



In [ ]:

for i in data['Drug'].unique():
    x = data.Na_to_K[data.Drug == i ].min()
    y=  data.Na_to_K[data.Drug == i ].max()
    print("min and max value of",i,x,y)
    

In [ ]:
#data preprocessing
data['Sex']=data['Sex'].replace({'M':0,'F':1})
data['Cholesterol']=data['Cholesterol'].replace({'NORMAL':0,'HIGH':1})
df=data.groupby('BP').size()/len(data)
data['BP']=data['BP'].map(df)
data['Na_to_K_Bigger_Than_15'] = [1 if i >=15.015 else 0 for i in data.Na_to_K]
data['Drug']=data['Drug'].replace({'DrugY':0,'drugC':1,'drugX':2,'drugA':3,'drugB':4})

cols = data.columns.tolist()
cols = [cols[-1]]+cols[:-1]
data=data.reindex(columns=cols)
data

In [ ]:
values=data.values
X, y = values[:, :-1], values[:, -1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.4, random_state = 2, stratify = y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import r2_score
drugTree = DecisionTreeClassifier(criterion = "entropy",max_depth = 4)
print("Training Set: ", X_train.shape, y_train.shape)
print("Testing Set: ", X_test.shape, y_test.shape)
drugTree.fit(X_train,y_train)
predTree = drugTree.predict(X_test)

print("The Accuracy Score is:", metrics.accuracy_score(y_test,predTree))
print("r^2 Score is",r2_score(y_test,predTree))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(random_state = 42 , oob_score=True)
accuracies = cross_val_score(rfc, X_train, y_train, cv=5)
rfc.fit(X_train,y_train)
pred = rfc.predict(X_test)
print("Train Score:",np.mean(accuracies))
print("Test Score:",rfc.score(X_test,y_test))
print("The Accuracy Score is:", metrics.accuracy_score(y_test,pred))
print("r^2 Score is",r2_score(y_test,pred))
print("Oobscore",rfc.score(X_test, y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
accuracies = cross_val_score(knn, X_train, y_train, cv=5)
knn.fit(X_train,y_train)

print("Train Score:",np.mean(accuracies))
print("Test Score:",knn.score(X_test,y_test))